# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [8]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [9]:
# to do: homework :)

In [12]:
import dlt
import duckdb

## 1. Use a generator

In [10]:
# Question 1: What is the sum of the outputs of the generator for limit = 5?
limit = 5
generator = square_root_generator(limit)
total_sum = 0
for sqrt_value in generator:
  total_sum += sqrt_value

print(total_sum)

8.382332347441762


In [11]:
# Question 2: What is the 13th number yielded
limit = 13
generator = square_root_generator(limit)
sqrt_values = [sqrt_value for sqrt_value in generator]

print(sqrt_values[12])

3.605551275463989


## 2. Append a generator to a table with existing data

### Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [14]:
# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destination as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_append",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707316720.1890304 is LOADED and contains no failed jobs


In [20]:
# show outcome
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the table
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data
print("\n\n\n people_append table below:")

people = conn.sql("SELECT * FROM people_append").df()
display(people)

print("\n\n\n sum of ages:")
sum_ages = conn.sql("SELECT SUM(Age) AS sum_ages FROM people_append").df()
display(sum_ages)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_append       │
└─────────────────────┘




 people_append table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707316720.1890304,0Ue8RNCdgpTzsw
1,2,Person_2,27,City_A,1707316720.1890304,FHTz0eulBWOr7A
2,3,Person_3,28,City_A,1707316720.1890304,XQsy9t8rBXHIDw
3,4,Person_4,29,City_A,1707316720.1890304,TSSKpFd5YLvsxw
4,5,Person_5,30,City_A,1707316720.1890304,+ZOVuCtYAXOZPA





 sum of ages:


,sum_ages
0,140.0


### Append the second generator to the same table. After correctly appending the data, calculate the sum of all ages of people.

In [21]:
info = generators_pipeline.run(people_2(),
										table_name="people_append",
										write_disposition="append")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707317309.159645 is LOADED and contains no failed jobs


In [22]:
print("\n\n\n people_append table below, with the data from the 2 generators:")
people = conn.sql("SELECT * FROM people_append").df()
display(people)

print("\n\n\n sum of ages after appending both generators:")
sum_ages = conn.sql("SELECT SUM(Age) AS sum_ages FROM people_append").df()
display(sum_ages)




 people_append table below, with the data from the 2 generators:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707316720.1890304,0Ue8RNCdgpTzsw,None
1,2,Person_2,27,City_A,1707316720.1890304,FHTz0eulBWOr7A,None
2,3,Person_3,28,City_A,1707316720.1890304,XQsy9t8rBXHIDw,None
3,4,Person_4,29,City_A,1707316720.1890304,TSSKpFd5YLvsxw,None
4,5,Person_5,30,City_A,1707316720.1890304,+ZOVuCtYAXOZPA,None
5,3,Person_3,33,City_B,1707317309.159645,NCyt9uYEaPNhzA,Job_3
6,4,Person_4,34,City_B,1707317309.159645,J9tIzQ4uv09GPA,Job_4
7,5,Person_5,35,City_B,1707317309.159645,i+2em/D8gG4UaQ,Job_5
8,6,Person_6,36,City_B,1707317309.159645,6YiDJdM6ClgePQ,Job_6
9,7,Person_7,37,City_B,1707317309.159645,ioiz5DwwXtYF8Q,Job_7





 sum of ages after appending both generators:


,sum_ages
0,353.0


## 3. Merge a generator

In [23]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key="ID")

print(info)

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print("\n\n\n people_merge table below, only with the data from the 1st generator")
people = conn.sql("SELECT * FROM people_merge").df()
display(people)

Pipeline dlt_colab_kernel_launcher load step completed in 0.46 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707317854.7676086 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id
0,5,Person_5,30,City_A,1707317854.7676086,dHEgS/BUi5JKeg
1,1,Person_1,26,City_A,1707317854.7676086,zoBKukSE803abA
2,2,Person_2,27,City_A,1707317854.7676086,r2oQ6EzRcVe6YQ
3,4,Person_4,29,City_A,1707317854.7676086,wHt3tEdlfRE/aQ
4,3,Person_3,28,City_A,1707317854.7676086,acoCUELE8IOYIQ


In [24]:
info = generators_pipeline.run(people_2(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key="ID")

print(info)
print("\n\n\n people_merge table below, with the data from both generators merged")
people = conn.sql("SELECT * FROM people_merge").df()
display(people)

Pipeline dlt_colab_kernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707317929.7517152 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707317854.7676086,zoBKukSE803abA,None
1,2,Person_2,27,City_A,1707317854.7676086,r2oQ6EzRcVe6YQ,None
2,5,Person_5,35,City_B,1707317929.7517152,ex0P/TpRpYwk2w,Job_5
3,8,Person_8,38,City_B,1707317929.7517152,PxXPk5oc74GcPA,Job_8
4,7,Person_7,37,City_B,1707317929.7517152,ckNUjw/tVwwyfw,Job_7
5,4,Person_4,34,City_B,1707317929.7517152,wjvbUt1JgPWr9g,Job_4
6,3,Person_3,33,City_B,1707317929.7517152,pCp4Z6IMi9UYqQ,Job_3
7,6,Person_6,36,City_B,1707317929.7517152,YcpnZYQUq+y2DQ,Job_6


In [25]:
print("\n\n\n sum of ages after merging both generators:")
sum_ages = conn.sql("SELECT SUM(Age) AS sum_ages FROM people_merge").df()
display(sum_ages)




 sum of ages after merging both generators:


,sum_ages
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX